In [315]:
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import numpy as np

In [294]:
eq_data_path = Path("../../data/raw/1/earthquake_data_tsunami.csv")
tsunami_data_path = Path("../../data/raw/1/tsunami_dataset.csv")

eq_df = pd.read_csv(eq_data_path)
tsunami_df = pd.read_csv(tsunami_data_path)

In [295]:
tsunami_df.head()

,ID,YEAR,MONTH,DAY,HOUR,MINUTE,LATITUDE,LONGITUDE,LOCATION_NAME,COUNTRY,...,CAUSE,EVENT_VALIDITY,EQ_MAGNITUDE,EQ_DEPTH,TS_INTENSITY,DAMAGE_TOTAL_DESCRIPTION,HOUSES_TOTAL_DESCRIPTION,DEATHS_TOTAL_DESCRIPTION,URL,COMMENTS
0,12,-330,NaN,NaN,NaN,NaN,40.00,25.000,"E. SPORADES ISLANDS, AEGEAN ISLANDS",GREECE,...,Earthquake,Very Doubtful Tsunami,7.0,NaN,NaN,NaN,NaN,NaN,https://www.ngdc.noaa.gov/hazel/view/hazards/t...,"330 B.C. Aegean Sea, Sporades Islands, 40 N 25..."
1,481,1764,2.0,11.0,NaN,NaN,51.45,-2.583,"BRISTOL, ENGLAND",UK,...,Unknown,Very Doubtful Tsunami,NaN,NaN,NaN,NaN,NaN,NaN,https://www.ngdc.noaa.gov/hazel/view/hazards/t...,"Reference #1894, in full: ""On Saturday the 11t..."
2,71,859,NaN,NaN,NaN,NaN,36.08,36.250,SAMANDAGI,TURKEY,...,Earthquake,Questionable Tsunami,NaN,NaN,3.0,NaN,NaN,NaN,https://www.ngdc.noaa.gov/hazel/view/hazards/t...,"859 (possibly 861), November. Levantian Sea, N..."
3,186,1580,1.0,NaN,NaN,NaN,40.00,128.000,YELLOW SEA,NORTH KOREA,...,Unknown,Very Doubtful Tsunami,NaN,NaN,1.0,NaN,NaN,NaN,https://www.ngdc.noaa.gov/hazel/view/hazards/t...,<P><blockquote><i>Reference #414:</i></blockqu...
4,5,-1300,NaN,NaN,NaN,NaN,39.96,26.240,"IONIAN COASTS, TROAD",TURKEY,...,Unknown,Questionable Tsunami,6.0,NaN,5.0,NaN,NaN,NaN,https://www.ngdc.noaa.gov/hazel/view/hazards/t...,1300 B.C. Ionian and Aegean Seas. References t...


In [296]:
min_year = eq_df['Year'].min()
min_year

2001

In [297]:
tsunami_df = tsunami_df[tsunami_df['YEAR']>2000]


In [298]:
print(tsunami_df["YEAR"].min())

2001


In [299]:
print("Dataset Shape:", tsunami_df.shape)
print("\nColumn Names:", tsunami_df.columns.tolist())
print("\nMissing Values:\n", tsunami_df.isnull().sum())
print("\nFirst 5 rows:\n", tsunami_df.head())
print("\nStatistical Summary:\n", tsunami_df.describe())

Dataset Shape: (246, 21)

Column Names: ['ID', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'LATITUDE', 'LONGITUDE', 'LOCATION_NAME', 'COUNTRY', 'REGION', 'CAUSE', 'EVENT_VALIDITY', 'EQ_MAGNITUDE', 'EQ_DEPTH', 'TS_INTENSITY', 'DAMAGE_TOTAL_DESCRIPTION', 'HOUSES_TOTAL_DESCRIPTION', 'DEATHS_TOTAL_DESCRIPTION', 'URL', 'COMMENTS']

Missing Values:
 ID                            0
YEAR                          0
MONTH                         0
DAY                           0
HOUR                         20
MINUTE                       20
LATITUDE                      0
LONGITUDE                     0
LOCATION_NAME                 0
COUNTRY                       0
REGION                        0
CAUSE                         0
EVENT_VALIDITY                0
EQ_MAGNITUDE                 35
EQ_DEPTH                     34
TS_INTENSITY                240
DAMAGE_TOTAL_DESCRIPTION    129
HOUSES_TOTAL_DESCRIPTION    187
DEATHS_TOTAL_DESCRIPTION    176
URL                           0
COMMENTS        

### Using year, month, latitude and longitude; we will match and merge two datasets

In [300]:
# tsunami_locations_and_info = tsunami_df[['YEAR', 'MONTH', 'LATITUDE', 'LONGITUDE', 'EQ_MAGNITUDE', 'EQ_DEPTH']].dropna()
# before_tsunami_count = (eq_df['tsunami'] == 1).sum()
# print(f"Tsunami değeri 1 olan deprem sayısı (güncelleme öncesi): {before_tsunami_count}")
# print(f"Tsunami değeri 0 olan deprem sayısı (güncelleme öncesi): {(eq_df['tsunami'] == 0).sum()}")
# # Her deprem için tsunami kontrolü yap
# for idx, row in eq_df.iterrows():

#     if eq_df.loc[idx, 'tsunami'] == 0:  # Sadece tsunami değeri 0 olanları kontrol et

#         same_time = tsunami_locations_and_info[
#             (tsunami_locations_and_info['YEAR'] == row['Year']) & 
#             (tsunami_locations_and_info['MONTH'] == row['Month'])
#         ]
        
#         if not same_time.empty:
#             lat_tolerance = 1.0
#             lon_tolerance = 1.0
#             magnitude_tolerance = 0.1
            
#             close_location = same_time[
#                 (abs(same_time['LATITUDE'] - row['latitude']) <= lat_tolerance) &
#                 (abs(same_time['LONGITUDE'] - row['longitude']) <= lon_tolerance) &
#                 (abs(same_time['EQ_MAGNITUDE'] - row['magnitude']) <= magnitude_tolerance)  # Magnitude kriteri eklendi
#             ]
            
#             if not close_location.empty:
#                 eq_df.loc[idx, 'tsunami'] = 1
#                 #print(f"Updated tsunami for index {idx}, Year: {row['Year']}, Month: {row['Month']}")
#                 magnitude_eq = row['magnitude']
#                 magnitude_ts = close_location.iloc[0]['EQ_MAGNITUDE']
#                 depth_eq = row['depth']
#                 depth_ts = close_location.iloc[0]['EQ_DEPTH']
                
# print(f"Tsunami değeri 1 olan deprem sayısı: {(eq_df['tsunami'] == 1).sum()}")
# print(f"Tsunami değeri 0 olan deprem sayısı: {(eq_df['tsunami'] == 0).sum()}")

# print(f"Depremelere yeni atanan tsunami sayısı: {(eq_df['tsunami'] == 1).sum() - before_tsunami_count}")
# print(f"Eliminated by magnitude/depth offset: {eliminated_by_offset}")

In [316]:
tsunami_locations_and_info = tsunami_df[['YEAR', 'MONTH', 'LATITUDE', 'LONGITUDE', 'EQ_MAGNITUDE', 'EQ_DEPTH']].dropna()
before_tsunami_count = (eq_df['tsunami'] == 1).sum()
newly_assigned = []

print(f"Tsunami değeri 1 olan deprem sayısı (güncelleme öncesi): {before_tsunami_count}")
print(f"Tsunami değeri 0 olan deprem sayısı (güncelleme öncesi): {(eq_df['tsunami'] == 0).sum()}")

Tsunami değeri 1 olan deprem sayısı (güncelleme öncesi): 431
Tsunami değeri 0 olan deprem sayısı (güncelleme öncesi): 351


In [ ]:


# Adaptive tolerances based on research
def get_adaptive_tolerances(magnitude, depth):
    """
    Returns adaptive tolerances based on magnitude and depth.
    Research shows larger events and deeper events have greater location/magnitude uncertainties.
    """
    # Latitude/Longitude tolerance (degrees)
    # Historical events and larger magnitudes have greater location uncertainty
    if magnitude >= 7.0:
        lat_tol = 2.0  # ~220 km at equator
        lon_tol = 2.0
    elif magnitude >= 6.0:
        lat_tol = 1.5  # ~165 km
        lon_tol = 1.5
    else:
        lat_tol = 1.0  # ~110 km
        lon_tol = 1.0
    
    # Magnitude tolerance
    # Research shows 0.3-0.6 typical, up to 1.0+ for great earthquakes
    if magnitude >= 7.5:
        mag_tol = 0.8  # Great earthquakes have larger discrepancies
    elif magnitude >= 6.5:
        mag_tol = 0.6
    else:
        mag_tol = 0.4  # Smaller events more consistent
    
    # Depth tolerance (km)
    # Deep earthquakes have much larger depth uncertainties
    if depth >= 300:
        depth_tol = 100  # Very deep events
    elif depth >= 100:
        depth_tol = 75   # Intermediate depth
    elif depth >= 50:
        depth_tol = 50
    else:
        depth_tol = 30   # Shallow events more precise
    
    return lat_tol, lon_tol, mag_tol, depth_tol

# Confidence scoring system
def calculate_match_confidence(lat_diff, lon_diff, mag_diff, depth_diff, 
                                lat_tol, lon_tol, mag_tol, depth_tol):
    """
    Calculate a confidence score for the match (0-100%)
    """
    # Normalized differences (0 = perfect match, 1 = at tolerance limit)
    lat_norm = min(lat_diff / lat_tol, 1.0)
    lon_norm = min(lon_diff / lon_tol, 1.0)
    mag_norm = min(mag_diff / mag_tol, 1.0)
    depth_norm = min(depth_diff / depth_tol, 1.0) if depth_tol > 0 else 0
    
    # Weighted confidence (location 40%, magnitude 40%, depth 20%)
    # Research shows magnitude and location are most critical for tsunami matching
    confidence = 100 * (1 - (0.2 * lat_norm + 0.2 * lon_norm + 
                             0.4 * mag_norm + 0.2 * depth_norm))
    return max(0, confidence)

# Time window tolerance - allow ±1 month for historical data inaccuracies
def check_time_match(eq_year, eq_month, ts_year, ts_month):
    """
    Check if times match within tolerance.
    Historical catalogs may have month discrepancies due to calendar conversions.
    """
    if eq_year != ts_year:
        return False
    # Allow exact month or ±1 month difference
    month_diff = abs(eq_month - ts_month)
    return month_diff <= 1

# Enhanced matching algorithm
for idx, row in eq_df.iterrows():
    if eq_df.loc[idx, 'tsunami'] == 0:
        # First filter by time (with tolerance)
        same_time = tsunami_locations_and_info[
            tsunami_locations_and_info.apply(
                lambda x: check_time_match(row['Year'], row['Month'], 
                                          x['YEAR'], x['MONTH']), 
                axis=1
            )
        ]
        
        if not same_time.empty:
            best_match = None
            best_confidence = 0
            
            for _, ts_event in same_time.iterrows():
                # Get adaptive tolerances
                avg_magnitude = (row['magnitude'] + ts_event['EQ_MAGNITUDE']) / 2
                avg_depth = (row['depth'] + ts_event['EQ_DEPTH']) / 2
                lat_tol, lon_tol, mag_tol, depth_tol = get_adaptive_tolerances(
                    avg_magnitude, avg_depth
                )
                
                # Calculate differences
                lat_diff = abs(ts_event['LATITUDE'] - row['latitude'])
                lon_diff = abs(ts_event['LONGITUDE'] - row['longitude'])
                mag_diff = abs(ts_event['EQ_MAGNITUDE'] - row['magnitude'])
                depth_diff = abs(ts_event['EQ_DEPTH'] - row['depth'])
                
                # Check if within tolerances
                if (lat_diff <= lat_tol and lon_diff <= lon_tol and 
                    mag_diff <= mag_tol and depth_diff <= depth_tol):
                    
                    # Calculate confidence
                    confidence = calculate_match_confidence(
                        lat_diff, lon_diff, mag_diff, depth_diff,
                        lat_tol, lon_tol, mag_tol, depth_tol
                    )
                    
                    # Keep best match
                    if confidence > best_confidence:
                        best_confidence = confidence
                        best_match = {
                            'ts_event': ts_event,
                            'lat_diff': lat_diff,
                            'lon_diff': lon_diff,
                            'mag_diff': mag_diff,
                            'depth_diff': depth_diff,
                            'confidence': confidence,
                            'tolerances': (lat_tol, lon_tol, mag_tol, depth_tol)
                        }
            
            # Assign tsunami if good match found (threshold: 50% confidence)
            if best_match and best_confidence >= 50:
                eq_df.loc[idx, 'tsunami'] = 1
                
                ts = best_match['ts_event']
                newly_assigned.append({
                    'index': idx,
                    'year': row['Year'],
                    'month': row['Month'],
                    'eq_lat': row['latitude'],
                    'eq_lon': row['longitude'],
                    'eq_magnitude': row['magnitude'],
                    'eq_depth': row['depth'],
                    'ts_lat': ts['LATITUDE'],
                    'ts_lon': ts['LONGITUDE'],
                    'ts_magnitude': ts['EQ_MAGNITUDE'],
                    'ts_depth': ts['EQ_DEPTH'],
                    'lat_diff': best_match['lat_diff'],
                    'lon_diff': best_match['lon_diff'],
                    'mag_diff': best_match['mag_diff'],
                    'depth_diff': best_match['depth_diff'],
                    'confidence': best_match['confidence'],
                    'lat_tol': best_match['tolerances'][0],
                    'lon_tol': best_match['tolerances'][1],
                    'mag_tol': best_match['tolerances'][2],
                    'depth_tol': best_match['tolerances'][3]
                })

after_tsunami_count = (eq_df['tsunami'] == 1).sum()

print(f"\nTsunami değeri 1 olan deprem sayısı (güncelleme sonrası): {after_tsunami_count}")
print(f"Tsunami değeri 0 olan deprem sayısı (güncelleme sonrası): {(eq_df['tsunami'] == 0).sum()}")
print(f"Depremelere yeni atanan tsunami sayısı: {after_tsunami_count - before_tsunami_count}")

# Enhanced reporting with confidence levels
if newly_assigned:
    print(f"\n=== YENİ ATANAN {len(newly_assigned)} DEPREMİN DETAYLARI ===")
    
    high_conf = [eq for eq in newly_assigned if eq['confidence'] >= 80]
    medium_conf = [eq for eq in newly_assigned if 60 <= eq['confidence'] < 80]
    low_conf = [eq for eq in newly_assigned if eq['confidence'] < 60]
    
    print(f"\nYüksek güvenilirlik (≥80%): {len(high_conf)} deprem")
    print(f"Orta güvenilirlik (60-80%): {len(medium_conf)} deprem")
    print(f"Düşük güvenilirlik (<60%): {len(low_conf)} deprem")
    
    print(f"\nOrtalama güvenilirlik: {np.mean([eq['confidence'] for eq in newly_assigned]):.1f}%")
    
    # Show top 10 matches
    print("\n=== EN İYİ 10 EŞLEŞME ===")
    sorted_matches = sorted(newly_assigned, key=lambda x: x['confidence'], reverse=True)[:10]
    for i, eq in enumerate(sorted_matches, 1):
        print(f"\n{i}. Eşleşme (Güvenilirlik: {eq['confidence']:.1f}%)")
        print(f"   Tarih: {eq['year']}/{eq['month']}")
        print(f"   Konum farkı: Δlat={eq['lat_diff']:.3f}° (tol={eq['lat_tol']}), "
              f"Δlon={eq['lon_diff']:.3f}° (tol={eq['lon_tol']})")
        print(f"   Büyüklük farkı: ΔM={eq['mag_diff']:.2f} (tol={eq['mag_tol']})")
        print(f"   Derinlik farkı: Δd={eq['depth_diff']:.1f} km (tol={eq['depth_tol']})")

    # Show bottom 10 matches
    print("\n=== EN KÖTÜ 10 EŞLEŞME ===")
    sorted_matches = sorted(newly_assigned, key=lambda x: x['confidence'])[:10]
    for i, eq in enumerate(sorted_matches, 1):
        print(f"\n{i}. Eşleşme (Güvenilirlik: {eq['confidence']:.1f}%)")
        print(f"   Tarih: {eq['year']}/{eq['month']}")
        print(f"   Konum farkı: Δlat={eq['lat_diff']:.3f}° (tol={eq['lat_tol']}), "
              f"Δlon={eq['lon_diff']:.3f}° (tol={eq['lon_tol']})")
        print(f"   Büyüklük farkı: ΔM={eq['mag_diff']:.2f} (tol={eq['mag_tol']})")
        print(f"   Derinlik farkı: Δd={eq['depth_diff']:.1f} km (tol={eq['depth_tol']})")

    print(len(newly_assigned['confidence']>85))

Tsunami değeri 1 olan deprem sayısı (güncelleme öncesi): 304
Tsunami değeri 0 olan deprem sayısı (güncelleme öncesi): 478

Tsunami değeri 1 olan deprem sayısı (güncelleme sonrası): 431
Tsunami değeri 0 olan deprem sayısı (güncelleme sonrası): 351
Depremelere yeni atanan tsunami sayısı: 127

=== YENİ ATANAN 127 DEPREMİN DETAYLARI ===

Yüksek güvenilirlik (≥80%): 114 deprem
Orta güvenilirlik (60-80%): 10 deprem
Düşük güvenilirlik (<60%): 3 deprem

Ortalama güvenilirlik: 93.7%

=== EN İYİ 10 EŞLEŞME ===

1. Eşleşme (Güvenilirlik: 100.0%)
   Tarih: 2015.0/11.0
   Konum farkı: Δlat=0.000° (tol=1.5), Δlon=0.000° (tol=1.5)
   Büyüklük farkı: ΔM=0.00 (tol=0.6)
   Derinlik farkı: Δd=0.0 km (tol=30)

2. Eşleşme (Güvenilirlik: 100.0%)
   Tarih: 2012.0/9.0
   Konum farkı: Δlat=0.000° (tol=2.0), Δlon=0.000° (tol=2.0)
   Büyüklük farkı: ΔM=0.00 (tol=0.8)
   Derinlik farkı: Δd=0.0 km (tol=30)

3. Eşleşme (Güvenilirlik: 100.0%)
   Tarih: 2012.0/8.0
   Konum farkı: Δlat=0.000° (tol=2.0), Δlon=0.000° (t

In [314]:
print(len([eq for eq in newly_assigned if eq['confidence'] > np.mean([eq['confidence'] for eq in newly_assigned])]))

82
